In [1]:
get_ipython().system(u' pip install --upgrade pip')
get_ipython().system(u' pip install beautifulsoup4')
!pip install lxml
!pip install html5lib
!pip install requests

#Importing packages
import requests
from bs4 import BeautifulSoup

     |████████████████████████████████| 1.5MB 6.9MB/s eta 0:00:01
  Found existing installation: pip 19.1.1
    Uninstalling pip-19.1.1:
      Successfully uninstalled pip-19.1.1


In [2]:
import pandas as pd

In [3]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [4]:
soup = BeautifulSoup(source, 'xml')

In [5]:
table=soup.find('table')

In [6]:
#Dataframe will consists of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [10]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [11]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Data Cleaning

In [12]:
df=df[df['Borough']!='Not assigned']

In [15]:

df[df['Neighborhood']=='Not assigned']

df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [17]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [18]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [19]:
df_merge.drop_duplicates(inplace=True)

In [24]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [25]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods"
3,M4A,North York,"Victoria Village, Victoria Village, Victoria V..."
6,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
9,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
12,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [26]:

df_merge.shape

(103, 3)